# Binary evolutions for the progenitor of J08408

Study of grid of binaries with a NS orbiting around a massive star. The aim of this notebook is to present a clear picture on the parameters which are related to J08408

## Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.optimize import bisect

# some constants
one_third = 1e0/3e0
pi = 3.1415926535897932384626433832795028841971693993751e0
standard_cgrav = 6.67428e-8
Msun = 1.9892e33
Rsun = 6.9598e10
Hubble_time = 13.461701658024014e0  # Gyr
clight = 2.99792458e10
secyer = 3.1558149984e7

# ---------------------------------------------
# Set some global options
np.seterr(all='ignore')

# ---------------------------------------------
# Load data
root = '..'
run_location = f'{root}/data/natal-kicks'
mesa_run_location = f'{root}/data'
config_location = f'{root}/config'

idgrid, Pgrid, agrid, egrid, probgrid = np.loadtxt(f'{run_location}/target_orbits.data', skiprows=9, unpack=True)

# load summary of mesa simulations
import pandas as pd

additional_info_on_progenitors = pd.read_csv(f'{mesa_run_location}/additional_info_of_star_plus_ns.data', delim_whitespace=True)
detail_progenitors = pd.read_csv(f'{mesa_run_location}/details_of_progenitors_of_j08408.data', delim_whitespace=True)

mpl_style = f'{config_location}/style.mpl'
paper_style = f'{config_location}/paper-style.mpl'

## Parameters at core collapse

In [ ]:
m1precc = 8.3525627823294126E+000
m2precc = 3.2637938645139933E+001
aprecc = 7.3604203503769526E+001
pprecc = 1.1424984631360623E+001
r1precc = 2.4346822488037647E+000
r2precc = 1.8891193348004055E+001
remnantmass = 1.6601196874643882E+000

## Kepler law & some auxiliary functions

In [ ]:
def a_to_P(separation, m1, m2):
    """Kepler law to go from separation to orbital period
    
    Parameters
    ----------
    a: binary separation in [Rsun]
    m1: mass of primary star in [Msun]
    m2: mass of secondary star in [Msun]

    Returns
    -------
    period: orbital period in [days]
    """
    separation = separation * Rsun  # in cm
    m1 = m1 * Msun
    m2 = m2 * Msun   # in g

    period = 2 * np.pi * np.sqrt(separation**3/(standard_cgrav*(m1+m2)))/(86400.0)
    return period

def P_to_a(period, m1, m2):
    """
    Binary separation from known period

    Parameters
    ----------
    P: binary period in [days]
    M1: mass of primary star in [Msun]
    M2: mass of secondary star in [Msun]

    Returns
    -------
    a: binary separation in [Rsun]
    """
    period = period * 24e0 * 3600e0  # in sec
    m1 = m1 * Msun; m2 = m2 * Msun   # in g
    tmp = standard_cgrav * (m1 + m2) * (period/(2*np.pi)) ** 2
    separation = np.power(tmp, one_third)
    return separation/Rsun

def rlobe(m1, m2, separation):
    """Roche lobe approximation for a star using Eggleton (1983)
    formula

    Parameters
    ----------
    m1: mass of primary star in [Msun]
    m2: mass of secondary star in [Msun]
    separation: binary separation in [Rsun]

    Returns
    -------
    RL: Roche lobe of primary star in [Rsun]
    """
    one_third = 1e0/3e0
    q = np.power(m1/m2, one_third)
    RL = 0.49e0 * q**2 / (0.6 * q**2 + np.log(1+q))
    RL = separation * RL

    return RL

## Eccentricity & orbital periods

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(2.7,2.7))
ax.set_xscale('log');

# axis labels
latex = '{\\rm orb}'
ax.set_xlabel(f'$P_{latex}$ [days]')
ax.set_ylabel('eccentricity');

# axis limits
ax.set_xlim(3, 2e3)
ax.set_ylim(0, 1);

# theoretical limits
ecc_max = 0.999
ecs = np.linspace(0, ecc_max)
ax.plot(
    a_to_P(aprecc/(1+ecs), m2precc, remnantmass), ecs,
    ls=':', c='black', zorder=99, lw=2,
)
ax.plot(
    a_to_P(aprecc/(1-ecs), m2precc, remnantmass), ecs,
    ls=':', c='black', zorder=99, lw=2,
);
    
# a = a_pre / (1 + e)
# => e = 1 - (a_pre / a)
porbs = np.logspace(np.log10(np.min(Pfull)), np.log10(np.max(Pfull)))
ecss = np.zeros(len(porbs))
for k,p in enumerate(porbs):
    a = P_to_a(p, m2precc, remnantmass)
    ecss[k] = 1 - aprecc / a

ax.fill_between(porbs, 0, ecss, color='C0')

# a = a_pre / (1 - e)
# => e = -1 + (a_pre / a)
porbs = np.logspace(np.log10(np.min(Pfull)), np.log10(np.max(Pfull)))
ecss = np.zeros(len(porbs))
for k,p in enumerate(porbs):
    a = P_to_a(p, m2precc, remnantmass)
    ecss[k] = - 1 + aprecc / a

ax.fill_between(porbs, 0, ecss, color='C0', zorder=99)
ax.fill_between(np.linspace(0,4.22), 0, 1.01, color='C0', zorder=99)

# theoretical limits
ecc_max = 0.999
ecs = np.linspace(0, ecc_max)
ax.plot(
    a_to_P(aprecc/(1+ecs), m2precc, remnantmass), ecs,
    ls=':', c='black', zorder=99, lw=2,
)
ax.plot(
    a_to_P(aprecc/(1-ecs), m2precc, remnantmass), ecs,
    ls=':', c='black', zorder=99, lw=2,
);

# annotations
ax.annotate('forbidden region', fontsize=8, xy=(80,0.4), color='white');

# plot actual randomly computed binary
ax.scatter(Pgrid, egrid, s=0.1, c='black', zorder=9);

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(2.7,2.7))
ax.set_xscale('log');

# axis labels
latex = '{\\rm orb}'
ax.set_xlabel(f'$P_{latex}$ [days]')
ax.set_ylabel('eccentricity');

# axis limits
ax.set_xlim(3, 2e3)
ax.set_ylim(0, 1);

# plot actual randomly computed binary
ax.scatter(Pgrid, egrid, s=0.1, c='black', zorder=9);

# colormap settings
bounds = np.arange(0,5,1)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
cm = plt.get_cmap('RdYlBu')

mask = additional_info_on_progenitors['MT_case'] == 'none'
x = additional_info_on_progenitors['porb'][mask]
y = additional_info_on_progenitors['e'][mask]
z = additional_info_on_progenitors['min_chi2'][mask]
ax.scatter(x, y, s=2, c=z, cmap=cm, norm=norm, alpha=0.9)

mask = additional_info_on_progenitors['MT_case'] != 'none'
x = additional_info_on_progenitors['porb'][mask]
y = additional_info_on_progenitors['e'][mask]
z = additional_info_on_progenitors['min_chi2'][mask]
ax.scatter(x, y, s=6, c=z, cmap=cm, norm=norm, alpha=0.9);